<a href="https://colab.research.google.com/github/josselinereyes841/edd_1358_2021/blob/master/09_diciembre_1358.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Array2D:

    def __init__(self,rows, cols, value):
        self.__cols = cols
        self.__rows = rows
        self.__array=[[value for x in range(self.__cols)] for y in range(self.__rows)]

    def to_string(self):
        [print("---",end="") for x in range(self.__cols)]
        print("")
        for ren in self.__array:
            print(ren)
        [print("---",end="") for x in range(self.__cols)]
        print("")

    def get_num_rows(self):
        return self.__rows

    def get_num_cols(self):
        return self.__cols

    def get_item(self,row,col):
    	try:
    		return self.__array[row][col]
    	except Exception as e:
    		pass
        

    def set_item( self , row , col , valor ):
        self.__array[row][col]=valor

    def clearing(self, valor=0):
        for ren in range(self.__rows):
            for col in range(self.__cols):
                self.__array[ren][col]=valor


class Stack:

    def __init__(self):
        self.__data = []
        self.__size = 0

    def pop(self):
    	self.__size -= 1
    	return self.__data.pop()

    def get_size(self):
        return self.__size

    def peak(self):
        if len(self.__data) > 0:
            return self.__data[-1]
        else:
            return None

    def push(self,value):
        self.__data.append(value)
        self.__size += 1
    def to_string(self):
    	cont = 1
    	for i in self.__data[::-1]:
    		if cont == 1:
    			print('-----')
    			print(f'|{i}| <---- Salida')
    			print('-----')
    		elif cont == self.__size:
    			print('-----')
    			print(f'|{i}| <---- Entrada')
    			print('-----')
    		else:
    			print('-----')
    			print(f'|{i}|')
    			print('-----')
    		cont +=1


class LaberintoADT:
	def __init__(self,archivo):

		self.__camino = Stack()

		self.__anterior = Stack()
		self.__actual = Stack()
		self.__siguiente = Stack()

		entrada = open(archivo,'rt')
		datos = entrada.readlines()
		self.__rens = int(datos.pop(0).strip())
		self.__cols = int(datos.pop(0).strip())
		self.__entrada = tuple(map(int,datos[0].strip().split(',')))


		datos.pop(0)

		self.__laberinto = Array2D(self.__rens,self.__cols,'1')
		for renglon in range(self.__rens):
			info_ren = datos[renglon].strip().split(',')
			for columna in range (self.__cols):
				self.__laberinto.set_item(renglon,columna,info_ren[columna])


	def entorno(self,ren,col):
		'''Saber en el orden izquierda, arriba, derecha, abajo si hay algún camino posible'''

		if (col-1) > -1 and  self.__laberinto.get_item(ren,col-1) in ['0','S','E'] and ((ren,col-1) != self.__anterior.peak() or self.numero_caminos(ren,col) == 1):
			return(ren,col-1)
		elif (ren-1) > -1  and self.__laberinto.get_item(ren-1,col) in ['0','S','E'] and ((ren-1,col) != self.__anterior.peak() or self.numero_caminos(ren,col) == 1):
			return(ren-1,col)
		elif col+1 <= self.__cols  and self.__laberinto.get_item(ren,col+1) in ['0','S','E'] and ((ren,col+1) != self.__anterior.peak() or self.numero_caminos(ren,col) == 1):
			return(ren,col+1)
		elif ren+1 <= self.__rens  and self.__laberinto.get_item(ren+1,col) in ['0','S','E'] and ((ren+1,col) != self.__anterior.peak() or self.numero_caminos(ren,col) == 1):
			return(ren+1,col)

	def numero_caminos(self,row,col):
		'''Nos dice cuantas posibles rutas hay en una coordenada'''
		CAMINOS = 0

		for j in [row-1, row+1]:
			try:
				if j == -1 or j > self.__rens:
					pass
				elif self.__laberinto.get_item(j,col) in ['0','S','E']  :
					CAMINOS += 1
			except Exception as e:
				pass
		for j in [col-1, col+1]:
			try:
				if j == -1 or j > self.__cols:
					pass				
				elif self.__laberinto.get_item(row,j) in ['0','S','E'] :
					CAMINOS += 1
			except Exception as e:
				pass				
		return CAMINOS

	def crear_camino(self):
		'''Crea el camino a seguir usando pilas'''

		self.__actual.push(self.__entrada) 

		self.__camino.push(self.__actual.peak())
		self.__siguiente.push( self.entorno(self.__actual.peak()[0],self.__actual.peak()[1]))

		if self.__siguiente.peak() == None:#En caso de que sólo esté la entrada sin pasillos
			return False

		for i in range(self.__rens*self.__cols):

			w = False
			self.__camino.push(self.__siguiente.peak())
			self.__anterior.push(self.__actual.peak())
			self.__actual.push(self.__siguiente.peak())
			self.__siguiente.push(self.entorno(self.__actual.peak()[0],self.__actual.peak()[1]))

			if self.__siguiente.peak() == None:
				return False #No hay salida 
			
			elif self.__laberinto.get_item(self.__siguiente.peak()[0],self.__siguiente.peak()[1]) == 'S':
				self.__camino.push(self.__siguiente.peak())
				return True #Sí hay salida

			while not w:
				try:
					if self.numero_caminos(self.__siguiente.peak()[0],self.__siguiente.peak()[1]) == 1 and self.__camino.get_size() > 1:
						self.__laberinto.set_item( self.__siguiente.peak()[0] , self.__siguiente.peak()[1] , 'x' )
						self.__camino.pop()
						self.__siguiente.pop()
						self.__actual.pop()
						self.__anterior.pop()
					else:
						w = True

				except Exception as e:
					pass

	def resolver(self):
		''' Muestra el camino o no en caso de que haya uno '''
		if self.crear_camino():
			self.mostrar_camino()
			self.__camino.pop()
			for i in range(self.__camino.get_size()):
				if self.__camino.get_size() == 1:
					pass
				else:
					self.__laberinto.set_item(self.__camino.peak()[0],self.__camino.peak()[1],'#')
					self.__camino.pop()
			self.__laberinto.to_string()

		else:
			print('Sin salida')

		
	def mostrar_camino(self):
		self.__camino.to_string()

	def to_string(self):
		self.__laberinto.to_string()


laberinto = LaberintoADT('laberintox.txt')
laberinto.resolver()

FileNotFoundError: ignored